In [6]:
!pip install -q kaggle
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle datasets download -d jangedoo/utkface-new
!unzip -qq utkface-new.zip

 99% 329M/331M [00:02<00:00, 104MB/s] 
100% 331M/331M [00:02<00:00, 124MB/s]


In [2]:
!pip install wandb

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/MyDrive/FaceAgeRegression

/content/drive/MyDrive/FaceAgeRegression


In [2]:

import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, Dataset
from tqdm import tqdm
import wandb
import torchvision.models as models
from data_loader import *


In [3]:
# wandb.init(project="Face Age Regression")
wandb.login()

wandb: Currently logged in as: nahid-ebrahimian (use `wandb login --relogin` to force relogin)


True

In [10]:
!wandb sweep config.yaml

wandb: Creating sweep from: config.yaml
wandb: Created sweep with ID: 6njvm2c2
wandb: View sweep at: https://wandb.ai/nahid-ebrahimian/Face-Age-Regression%28parameters%20setting%29/sweeps/6njvm2c2
wandb: Run sweep agent with: wandb agent "nahid-ebrahimian/Face-Age-Regression(parameters setting)/6njvm2c2"


### train

In [12]:
def train():
    # Initialize a new wandb run
    with wandb.init():
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        train_data = build_dataset(config.batch_size)
        model = build_model()
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)

        for epoch in range(10):
            total_loss = train_epoch(model, train_data, optimizer)
            print(f"Epoch: {epoch+1}, Loss: {total_loss}")
            if epoch %2 == 0:
              wandb.log({"loss": total_loss, "epochs": epoch}) 

In [13]:
def build_optimizer(network, optimizer, learning_rate):
    
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

def build_model():

  model = models.resnet50(pretrained=True)
  in_fetures = model.fc.in_features
  model.fc = nn.Linear(in_fetures, 1)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  # This freezes layers 1-6 in the total 10 layers of Resnet50
  ct = 0
  for child in model.children():
      ct += 1
      if ct < 7:
          for param in child.parameters():
              param.requires_grad = False
  return model

def build_dataset(batch_size):
  
  train_data = data_loader("/content/crop_part1")
  torch.manual_seed(0)
  train_dataset_size = int(0.8 * len(train_data))
  test_dataset_size = len(train_data) - train_dataset_size

  train_data, _ = torch.utils.data.random_split(train_data, [train_dataset_size, test_dataset_size])
  train_data = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)

  return train_data

In [14]:
loss_function = nn.L1Loss()

In [15]:
# train

def train_epoch(model, train_data, optimizer):
  
    train_loss = 0.0
    for images, labels in tqdm(train_data):
      device = torch.device("cuda")
      images, labels = images.to(device), labels.to(device)

      optimizer.zero_grad()
      images = images.float()
      preds = model(images)

      loss = loss_function(preds,  labels.float())
      loss.backward()
      optimizer.step()
      train_loss += loss

    total_loss = train_loss / len(train_data)
    return train_loss

In [ ]:
wandb.agent("6njvm2c2", train)
# wandb.agent("8mriggqw", train, project="Face-Age-Regression(parameters setting)")


wandb: Agent Starting Run: 2arc7r05 with config:
wandb: 	batch_size: 128
wandb: 	epochs: 10
wandb: 	learning_rate: 0.005819311835533811
wandb: 	optimizer: sgd


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 48/48 [01:10<00:00,  1.48s/it]


Epoch: 1, Loss: 776.531005859375


100%|██████████| 48/48 [01:11<00:00,  1.48s/it]


Epoch: 2, Loss: 572.1531982421875


...

100%|██████████| 48/48 [01:11<00:00,  1.49s/it]


Epoch: 7, Loss: 514.8695068359375


100%|██████████| 48/48 [01:11<00:00,  1.49s/it]


Epoch: 8, Loss: 497.49151611328125


 29%|██▉       | 14/48 [00:20<00:51,  1.51s/it]

In [ ]:
torch.save(model.state_dict(), "age_regression.pth")